Vamos a entrenar una CNN, esta a vez a Color y con el ds CIFAR-10.

In [2]:
# Arranque: imports, dataset y primer batch (MNIST)
import torch, torch.nn as nn, torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt


torch.manual_seed(3)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Aqui convertimos las imagenes a tensores
# Es decir, el valor de los pixeles pasa a estar entre 0 y 1
# y el shape de las imagenes pasa a ser (C,H,W) (Canales, Alto, Ancho)

transform = transforms.ToTensor()  # [0,1], shape (C,H,W)




train = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
test = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

loader_train = DataLoader(train, batch_size=128, shuffle=True)
loader_test = DataLoader(test, batch_size=128, shuffle=True)


images, labels = next(iter(loader_train))
print(images.shape, labels.shape)





cuda
torch.Size([128, 3, 32, 32]) torch.Size([128])


## Normalización "manual"

Vamos a Normalizar los canales, esto es una buena practica y es común hacerlo.

Se cargan todas las 50 000 imágenes de entrenamiento. (quedan 10.000 para test)

Cada imagen se convierte a valores entre 0 y 1. (lo hicimos previamente con ToTensor)

Ahora:

Para cada canal (R, G, B) se calcula:

media → el promedio de todos los valores de ese canal en todas las imágenes;

desviación típica → cuánto varían esos valores respecto a la media.

Tras eso se realiza un transform para normalizar los datos.

In [ ]:
#Paso 1: Convertir a float64 para que los calculos sean exactos


#En PyTorch, cada tensor tiene dimensiones (también llamadas axes o dims).
#Por ejemplo, un tensor de 3 dimensiones (3D) tiene forma (B, C, H, W)
#Donde:
#B = Batch size (numero de imagenes en el batch)
#C = Canales (RGB, 3)
#H = Alto
#W = Ancho

# 2. Inicializamos acumuladores
#Son tensores 1D con tantos elementos como canales (R, G, B)
# Seria algo como [0,0,0] con una precision de 64 bits para sus decimales
sum_c = torch.zeros(3, dtype=torch.float64)
sum_sq_c = torch.zeros(3, dtype=torch.float64)
num_pixels = 0

# 3. Bucle sobre cada batch
for batch, _ in loader_train:
    i = 0
    #Convertirmos  ese batch a float64 tambien
    x = batch.to(torch.float64)
    #Aseguramos que el shape sea (128, 3, 32, 32)
    b, c, h, w = x.shape

    #Para entender esta linea, revisa 01-Tensor_Fundamentals.ipynb seccion "Tensores aplicados a imagenes"
    #En resumen, sumamos todos los pixeles de cada canal de cada imagen en el batch y eso nos da un tensor de 3 elementos por imagen en el batch
    # con todos los pixeles sumados por canal (R, G, B)
    sum_c += x.sum(dim=(0, 2, 3))
    assert sum_c.shape == (3,)


    #Elevamos al cuadrado cada pixel de cada canal de cada imagen en el batch
    # y sumamos todos los pixeles de cada canal de cada imagen en el batch
    # esto es para calcular la varianza de los pixeles de cada canal de cada imagen en el batch
    sum_sq_c += (x ** 2).sum(dim=(0, 2, 3))
    assert sum_sq_c.shape == (3,)
    

    #Contamos cuantos pixeles llevamos acumulados
    num_pixels += b * h * w
    
    ############################# BUCLE TERMINADO ##############################

#Calculamos la media de los pixeles de cada canal de cada imagen en el batch
mean = sum_c / num_pixels
assert mean.shape == (3,)

#Calculamos la desviacion estandar de los pixeles de cada canal de cada imagen en el batch
std = ((sum_sq_c / num_pixels) - mean**2).sqrt()
assert std.shape == (3,)

print("mean:", mean)
print("std :", std)

#Creamos un transform que normaliza los pixeles de cada canal de cada imagen en el batch
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean.tolist(), std.tolist())
])

# Re-creamos los datasets con el nuevo transform
train = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
test = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

#Cargamos los datos en batches
loader_train = DataLoader(train, batch_size=128, shuffle=True)
loader_test = DataLoader(test, batch_size=128, shuffle=True)

#Como vemos hemos normalizado tanto el train como el test set, sin embargo, no calculamos mean y std de test set
#Esto es porque no queremos que el modelo vea datos de test set durante el entrenamiento
#Es decir, el modelo solo vera los datos de train set durante el entrenamiento 
#Y al final, evaluaremos el modelo con los datos de test set

#ref https://stats.stackexchange.com/questions/495357/why-do-we-normalize-test-data-on-the-parameters-of-the-training-data
#ref medium.com/%40spinjosovsky/normalize-data-before-or-after-split-of-training-and-testing-data-7b8005f81e26

#Ahora vamos a comprobar que los datos estan normalizados
images, labels = next(iter(loader_train))
print(images.shape, labels.shape)

mean_check = images.mean(dim=(0, 2, 3))
std_check  = images.std(dim=(0, 2, 3))

print("Mean (debería estar cerca de 0):", mean_check)
print("Std (debería estar cerca de 1):", std_check)


mean: tensor([0.4914, 0.4822, 0.4465], dtype=torch.float64)
std : tensor([0.2470, 0.2435, 0.2616], dtype=torch.float64)
torch.Size([128, 3, 32, 32]) torch.Size([128])
Mean (debería estar cerca de 0): tensor([-0.0497, -0.0542, -0.0447])
Std (debería estar cerca de 1): tensor([1.0260, 1.0227, 0.9957])


| Parámetro    | Valor                                                      |
| ------------ | ---------------------------------------------------------- | 
| Mean ≈ -0.05 |  Correcto — centrado en 0 con ligera desviación por batch | 
| Std ≈ 1.0    |  Perfecto — escala normalizada                            |                
| Ejecución    |  Correcta — sin renormalizaciones acumuladas              |                
